<a href="https://colab.research.google.com/github/ask-1407/my_portfolio/blob/main/NLP_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://kagglenote.com/kaggle/simpletransformers-claassification/

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install simpletransformers
!pip install mojimoji
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7
!pip install textcleaner

In [2]:
import pandas as pd
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from sklearn.metrics import accuracy_score
import mojimoji
import MeCab
import requests
import collections

In [58]:
PATH = '/content/drive/My Drive/Colab Notebooks/nlp_data/train.csv'

data = pd.read_csv(PATH)
drop_col = ['id','source']
data = data.drop(drop_col, axis=1)

test_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/nlp_data/test.csv')
test_data = test_data.drop(test_data.columns[1], axis=1) # sourceを削除

In [59]:
#前処理
data['text'] = data['text'].replace('\n', '', regex=True)# 改行コードを消去

# data.text=data.text.apply(lambda x: re.sub(r'[^A-Za-z]+',' ',x))
data.text=data.text.apply(lambda x: x.lower())#大文字を小文字へ
data.text= data.text.apply(mojimoji.han_to_zen) # 半角を全角へ
data.text=data.text.apply(lambda x:x.strip())


test_data.text=test_data.text.apply(lambda x: x.lower())#大文字を小文字へ
test_data.text=test_data.text.apply(mojimoji.han_to_zen) # 半角を全角へ
test_data.text=test_data.text.apply(lambda x:x.strip())


# data.replace( stopwords, '', regex=True) #改行コードを除去 
# data.to_csv('/content/drive/My Drive/Colab Notebooks/nlp_data/hoge.csv')

In [40]:
test_data

,id,text
0,001026808,上でも言ったけどオタクレベルの知識求めてる訳じゃない\nただ囲碁やります！って人が誰１人プロ...
1,00465ac96,たとえば、黒人なんかは、生物学的欠陥はないのに、文化的要因で、悪循環に陥り、実力をつけられず...
2,004674725,そうなんやろなあ色々と勿体ない感じしたわ\n終わり方と黒幕キャラは好きやったで\n\nちなワ...
3,00474460f,法的というか自治体ごとにバラバラの条例で定めてるだけだからな\n普通の淫行条例だと「青少年に...
4,004a7525c,別のジャーナリストの感想として言われてるので客観的な事実とは言えないけど、\n現地は不測の事...
...,...,...
3218,ffc4647ac,１人がいいのか？\nなんで変なのと同棲したのか…\nなにがしたいんだ…
3219,ffc6554ba,ロシアもだなあ\n元々北朝鮮はロシアの工作で作られた国だから
3220,ffd3b29c2,クネが国境に拡声器を設置して昼も夜も北の悪口鳴らしてんだとよ\nお互い当たらないように大砲撃...
3221,ffd3c69b6,当然って言い方が腹立つんだよなあ\r\nその時点で何か男より優位に立ちたいみたいな感じがして...


In [9]:
 tagger = MeCab.Tagger()
 
 def tokenize(text, stop_words):
     node = tagger.parseToNode(text)
     result = []
     while node:
         features = node.feature.split(',')
         
         if features[0] != 'BOS/EOS':
             if features[0] not in ['助詞', '助動詞']:
                 token = features[6] if features[6] != '*' else node.surface
                 result.append(token)
         
         node = node.next
     
     return result

In [45]:
 stop_words =['a','∀','!','#','$','%','&','・','\(','\)','\?','\+','\n','ｗ','￣','\＼','|ﾟヮﾟﾉ']

# i = 0
# j = 0

# for i in range(len(data['text'])):
#   data['text'][i] = tokenize(data['text'][i], stop_words)
  
# for j in range(len(test_data['text'])):
#   test_data['text'][j] = tokenize(test_data['text'][j], stop_words)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [61]:
stop_words =['a','∀','!','#','$','%','&','・','\(','\)','\?','\+','\n','ｗ','￣','\＼','|ﾟヮﾟﾉ']

import textcleaner as tc

print(data.text.remove_stpwrds)


ModuleNotFoundError: ignored

In [63]:
!pip install textcleaner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for textcleaner: filename=textcleaner-0.4.26-py3-none-any.whl size=4737 sha256=b565c80de173832ac4422744084096da642e81c683d7ea502ed0dc74be92557d
  Stored in directory: /root/.cache/pip/wheels/67/e1/1a/8a3dc9c64347e08039c1e69924c9af6ba0a395a0ed92cfb493
Successfully built textcleaner


In [60]:
train_df,eval_df=train_test_split(data,test_size=0.1,shuffle=0.3,random_state=0)
train_df, eval_df

(                                                   text  label
 464             一年戦争はもうボロボロ陸ガン自体は好きだけど、つじつま合わせは流石に無いと思う      0
 1270                     ×職歴が人の価値＆＃９８９８；＆＃６５０３８；職歴無しはカス      0
 1974              要領よくいかないとだめだろ俺なんて普段からお中元、お歳暮バンバン贈ってるぜ      0
 1853  たとえば、「消費税増税しない。議論もしない」といってたのに、増税法案を通しちゃったよね。その...      0
 2259                守備はええし怪我はないし足もあるからなハムで１番替えのきかない選手やろ      0
 ...                                                 ...    ...
 4931                    だって払わないとモンスターと戦う権利すらないんだぞ仕方ないだろ      0
 3264  そもそもコネ入学使えなくなったから制服で振り落としたろ！っていうアレなのに金のことばっか騒い...      0
 1653  せやで日本に暮らしてる在日韓国人・朝鮮人の数は全体の０．６％それに対して苗字左右対称の人の数...      0
 2607  大学は野郎共とバイクで遊んだりクラブやら行ったり女の子らと酒飲んだり海行ったりして楽しかった...      0
 2732   ｎｏｔｅって言うあんま知られてないｓｎｓで愚痴って拡散これ拡散したのは案外関係者かも知れんなあｗ      0
 
 [4730 rows x 2 columns],
                                                    text  label
 3035  他の事件でも呼び出しくらうようになるのはそんな気にしていないんだけど・・・そういう問題ではな...      0
 138               掻きまくってる掻きまくってたら変な汁が出て来てその汁で余計に痒くなる悪循環      0
 629        

In [64]:
#学習用パラメータ
params = {
    "max_seq_length": 128,#シークエンスの最大長。この値が大きいと文章の表現力が高くなってより高い精度が期待できる。大きすぎるとoverfittingや学習時間の増大などが生じる可能性がある。
    "train_batch_size": 64,#訓練時のバッチサイズ
    "eval_batch_size": 32,
    "num_train_epochs": 10,
    "use_early_stopping": True,
    "learning_rate": 1e-5,
    "n_gpu": 4,
    "overwrite_output_dir":True
}
weight = 1/ pd.DataFrame((train_df.label).tolist()).reset_index().groupby(0).count().values#不均衡データに対処するためのパラメーター
weight_sum = weight.sum()
weight /= weight_sum


model = ClassificationModel('roberta', 'roberta-base', args=params, num_labels=2, use_cuda=False)
model.train_model(train_df)

result, model_outputs, wrong_predictions = model.eval_model(eval_df)

y_pred, raw_outputs = model.predict(eval_df.text.tolist())
print(accuracy_score(y_pred, eval_df.label))

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

  0%|          | 0/4730 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/74 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/74 [00:00<?, ?it/s]

...

  0%|          | 0/526 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/17 [00:00<?, ?it/s]

  0%|          | 0/526 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

0.9505703422053232


In [66]:
test_data

,id,text
0,001026808,上でも言ったけどオタクレベルの知識求めてる訳じゃない\nただ囲碁やります！って人が誰１人プロ...
1,00465ac96,たとえば、黒人なんかは、生物学的欠陥はないのに、文化的要因で、悪循環に陥り、実力をつけられず...
2,004674725,そうなんやろなあ色々と勿体ない感じしたわ\n終わり方と黒幕キャラは好きやったで\n\nちなワ...
3,00474460f,法的というか自治体ごとにバラバラの条例で定めてるだけだからな\n普通の淫行条例だと「青少年に...
4,004a7525c,別のジャーナリストの感想として言われてるので客観的な事実とは言えないけど、\n現地は不測の事...
...,...,...
3218,ffc4647ac,１人がいいのか？\nなんで変なのと同棲したのか…\nなにがしたいんだ…
3219,ffc6554ba,ロシアもだなあ\n元々北朝鮮はロシアの工作で作られた国だから
3220,ffd3b29c2,クネが国境に拡声器を設置して昼も夜も北の悪口鳴らしてんだとよ\nお互い当たらないように大砲撃...
3221,ffd3c69b6,当然って言い方が腹立つんだよなあ\r\nその時点で何か男より優位に立ちたいみたいな感じがして...


In [65]:
predictions, raw_output =model.predict(test_data['text'].values.tolist())


  0%|          | 0/3223 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

In [67]:
### 結果の出力
result = pd.Series(predictions)
result = pd.DataFrame(result,columns=['label'])
result = pd.concat([test_data['id'],result],axis=1)
result.to_csv('/content/drive/My Drive/Colab Notebooks/nlp_data/result_bert.csv')

In [70]:
pred_df = pd.concat([result,pd.DataFrame(test_data['text'],columns=['text'])],axis=1)
# pred_df

In [71]:
sum(pred_df['label'])

301

In [53]:
pred_df.loc[pred_df['label'] == 1,['text','label']]

,text,label
37,"[日本人, 怒る, てる, ん, 。, 日本人, 生まれながら, 卑怯, 者, 、, 裏切り...",1
184,"[日本人, 金, 払う, トイレ, 行く, 奴, 行く, 状況, する, せる, 底辺, 家...",1
265,"[日本人, 才能, ある, 努力, する, 必要, 全く, ない, 、, チョン, すべて,...",1
288,"[女性, 交友, 関係, 分かる, 上手い, 付き合う, 言う, 大人, なる, 昔, 付き...",1
293,"[日本人, 拒否, 反応, これ, 思う, 一, 枚]",1
403,"[人生, 二, 回, 寝取る, れる, 。, 女, いつか, 俺, 離れる, 考える, られ...",1
421,"[日本人, そういう, 卑怯, こと, 言う, それ, いえる, 国民, 性, 信じる, ら...",1
517,"[本人, 報道, 否定, する, こと, そういう, こと, 今回, 会談, 自体, そう]",1
586,"[朝鮮, 人, 、, 何, 選挙, 妨害, 何, 選挙, 妨害, 判断, つく, 。, それ...",1
604,"[人生, 麻雀, ん, 配, 牌, 決まる, てる, しれる, ツモ, くる, 牌, 運, ...",1
